In [ ]:
import pandas as pd
import datetime

df = pd.read_excel('Отчет_доставки_по_Курьерам_14_06_2022_16_06_22.xlsx')
df = df[(df['Тип заказа']!='На время самовывоз') & (df['Тип заказа']!='Доставка самовывоз') & (df['Тип заказа']!='Самовывоз')]

for i in df['Курьер'].unique():
    print('Курьер: ', i)
    df_courier = df[df['Курьер'] == i]
    orders = df_courier.shape[0]
    print('Отвезенных заказов всего: ', orders)
    dovozi = df_courier[(df_courier['Сумма со скидкой, р.'] < 200) & (df_courier['Сумма со скидкой, р.'] > 0)]
    dovozi_amount = 0    

    for z in range(dovozi.shape[0]):
        if df_courier[(df_courier['Адрес'] == dovozi.iloc[z]['Адрес']) \
        & (df_courier['Фактическое время доставки'] <= (dovozi.iloc[z]['Фактическое время доставки']) + datetime.timedelta(minutes=3)) \
        & (df_courier['Фактическое время доставки'] >= (dovozi.iloc[z]['Фактическое время доставки']) - datetime.timedelta(minutes=3))].shape[0] > 1:
            df_courier = df_courier[df_courier['Номер чека']!= dovozi.iloc[z]['Номер чека']]
            dovozi_amount = dovozi_amount + 1
    
    print('Количество довозов без оплаты: ', dovozi_amount)
    orders_next = df_courier.shape[0]
    print('Отвезенных заказов без довозов: ', orders_next)
    
    arr = []
    for j in range(orders_next):
        reis = df_courier[(df_courier['Время отправки доставки'] < (df_courier.iloc[j]['Время отправки доставки'] + datetime.timedelta(minutes=20))) \
        & (df_courier['Время отправки доставки'] > (df_courier.iloc[j]['Время отправки доставки'] - datetime.timedelta(minutes=20)))].shape[0]
        arr = arr + [reis]  
    
    dict_ = {}
    for q in set(arr):
        dict_[q] = arr.count(q)/q 
    dict_ = dict(sorted(dict_.items()))
    print(dict_)

    print('Среднее количество заказов в рейсе: ',  orders_next / sum(dict_.values()))